# Data Availability

In [3]:
from huggingface_hub import HfApi
import pandas as pd
import itertools
import re
import requests
from bs4 import BeautifulSoup
import time
import json
import os
from huggingface_hub.utils import logging

from tags import * # tags.py
logging.set_verbosity_error()

In [4]:
api = HfApi()

## LLMs

### Information extraction

In [ ]:
models = api.list_models(full=True)

In [ ]:
model = itertools.islice(models, 0, 1000)
models_df = pd.DataFrame(model)
models_df.head(10)

In [ ]:
print(models_df.columns)

In [ ]:
# tag examples
models_df.loc[0]['tags']

In [ ]:
# richer tag information example
example_model = api.list_models(model_name='albert_bert_summarization_cnn_dailymail')
example_df = pd.DataFrame(example_model)
example_df.loc[0]['tags']

In [8]:
# Scrape languages from HF

url_languages = 'https://huggingface.co/languages'

response = requests.get(url_languages)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

code_tags = soup.find_all('code')
tag_language = [code_tag.get_text() for code_tag in code_tags]
print(tag_language)

tag_language.remove('jax') # 'jax' is the ISO for Jambi Malay (present in 3 datasets, 36 models), impossible to distinguish from JAX the library... TODO: better solution?

['en', 'zh', 'fr', 'es', 'ru', 'de', 'ja', 'pt', 'ko', 'ar', 'it', 'vi', 'tr', 'hi', 'id', 'pl', 'nl', 'th', 'cs', 'bn', 'fa', 'sv', 'ro', 'ca', 'fi', 'ta', 'da', 'hu', 'uk', 'ind', 'el', 'te', 'ur', 'bg', 'he', 'ml', 'ms', 'sl', 'mr', 'sw', 'sk', 'et', 'eu', 'kn', 'gu', 'sr', 'hr', 'no', 'lt', 'lv', 'pa', 'is', 'yo', 'am', 'vie', 'ne', 'az', 'af', 'ga', 'mt', 'si', 'gl', 'sq', 'or', 'kk', 'cy', 'tl', 'ceb', 'tha', 'as', 'mk', 'ha', 'hy', 'ka', 'my', 'uz', 'eng', 'ig', 'eo', 'be', 'nb', 'km', 'mn', 'ky', 'la', 'so', 'zu', 'min', 'jav', 'xh', 'ps', 'nn', 'rw', 'jv', 'yue', 'mya', 'br', 'tt', 'bs', 'ckb', 'lg', 'sa', 'wo', 'lo', 'ku', 'ug', 'sd', 'ilo', 'tw', 'ast', 'sun', 'tg', 'ace', 'lb', 'nso', 'gd', 'war', 'fil', 'oc', 'su', 'tk', 'fy', 'bug', 'tgl', 'gn', 'bjn', 'khm', 'sn', 'yi', 'ht', 'mai', 'ba', 'bo', 'tn', 'zlm', 'ban', 'fo', 'dv', 'kab', 'ln', 'bm', 'hin', 'ny', 'cv', 'ti', 'aa', 'shn', 'mg', 'sat', 'mi', 'arz', 'sah', 'lao', 'st', 'ee', 'mar', 'ia', 'pag', 'qu', 'hsb', 'ab',

In [9]:
# pattern matching functions

def extract_name(full_name):
    pattern = re.compile(r'[^/]+/(.+)')
    match = re.search(pattern, full_name)
    if match:
        return match.group(1) # the part after '/' might also contain version and number of parameters (impossible to extract in a uniform way)
    else:
        return full_name

def match_string(entries, pattern):
    pattern = re.compile(pattern)
    for entry in entries:
        match = pattern.match(entry)
        if match:
            return match.group(1)
    return None

def find_all_matches(entries, pattern):
    pattern = re.compile(pattern)
    matches = []
    for entry in entries:
        match = pattern.match(entry)
        if match:
            matches.append(match.group(1))
    return matches

def match_license(entries):
    return match_string(entries, r'license:(\S+)')

def match_dataset(entries):
    return find_all_matches(entries, r'dataset:(\S+)')

def match_uri(entries):
    uri = match_string(entries, r'arxiv:(\S+)')
    if uri is None:
        uri = match_string(entries, r'doi:(\S+)')
    return uri

In [ ]:
def extract_model_attributes(model_idx):

	model = models_df.loc[model_idx]
	model_tags = models_df.loc[model_idx]['tags']
	model_card_data = None
	try:
		model_card_data = next(api.list_models(model_name=model['id'], full=True, cardData=True)).card_data.to_dict()
	except AttributeError:
		print('No card data available for this model')
	model_attributes = dict()

	model_attributes['name'] = extract_name(model['id'])
	model_attributes['version'] = None # sometimes in model['id'] but impossible to extract in a uniform way
	model_attributes['numberOfParameters'] = None # sometimes in model['id'] or model description but impossible to extract in a uniform way

	model_attributes['quantization'] = None
	for t in model_tags:
		if t in tag_quantization:
			model_attributes['quantization'] = t

	model_attributes['architecture'] = None
	try:
		if model_card_data is not None:
			model_attributes['architecture'] = model_card_data['base_model']
	except KeyError:
		print('No architecture data available for this model')

	model_attributes['languages'] = []
	for t in model_tags:
		if t in tag_language:
			model_attributes['languages'].append(t)

	model_attributes['modelCreator'] = None # TODO: if base_model exists, look for 'author' of the base model
	try:
		if model_card_data is not None:
			base_model = model_card_data['base_model']
			base_model_data = pd.DataFrame(api.list_models(model_name=base_model, full=True))
			model_attributes['modelCreator'] = base_model_data.loc[0]['author']
	except KeyError:
		print('No base model data available for this model')

	model_attributes['licenseToUse'] = match_license(model_tags)

	model_attributes['libraryFramework'] = [] # TODO: change type into list(str) in our model
	for t in model_tags:
		if t in tag_library:
			model_attributes['libraryFramework'].append(t)

	model_attributes['contextLength'] = None
	model_attributes['developers'] = [model['author']]
	model_attributes['openSource'] = True

	model_attributes['uri'] = match_uri(model_tags)

	model_attributes['fineTuned'] = None # if there is a 'base_model' in card_data, it is fine-tuned
	try:
		if model_card_data is not None:
			if 'base_model' in model_card_data:
				model_attributes['fineTuned'] = True
	except KeyError:
		print('No base model data available for this model')

	model_attributes['carbonEmission [CO2eq tons]'] = None
	try:
		if model_card_data is not None:
			model_attributes['carbonEmission [CO2eq tons]'] = model_card_data['co2_eq_emissions']
	except KeyError:
		print('No emission data available for this model')

	model_attributes['tokenizer'] = None

	return model_attributes

In [ ]:
def model_to_json(model_idx):
    
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)
    
    model_attributes = extract_model_attributes(model_idx)
    
    with open(os.path.join(result_path, 'test_models.json'), 'w', encoding='utf-8') as f:
        json.dump(model_attributes, f, indent=4)

In [ ]:
def models_to_json(models_df):
    
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)

    output = []
    
    for model_idx in range(models_df.shape[0]):
        output.append(extract_model_attributes(model_idx))
    
    with open(os.path.join(result_path, 'ChatIMPACT.LargeLanguageModel.json'), 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=4)

In [ ]:
# models_to_json(models_df)

### Info extraction optimized

In [ ]:
def extract_model_attributes_optimized(model):
	model_tags = model.tags
	if model.card_data is not None:
		model_card_data = model.card_data.to_dict()
	else:
		model_card_data = None
	model_attributes = dict()

	model_attributes['name'] = extract_name(model.id)
	model_attributes['id'] = model.id
	model_attributes['version'] = None # sometimes in model['id'] but impossible to extract in a uniform way
	model_attributes['numberOfParameters'] = None # sometimes in model['id'] or model description but impossible to extract in a uniform way

	model_attributes['quantization'] = None
	for t in model_tags:
		if t in tag_quantization:
			model_attributes['quantization'] = t

	model_attributes['architecture'] = None
	try:
		if model_card_data is not None:
			model_attributes['architecture'] = model_card_data['base_model']
	except KeyError:
		pass

	model_attributes['languages'] = []
	for t in model_tags:
		if t in tag_language:
			model_attributes['languages'].append(t)

	model_attributes['modelCreator'] = None # TODO: if base_model exists, look for 'author' of the base model
	# try:
	# 	if model_card_data is not None:
	# 		if 'base_model' in model_card_data:
	# 			base_model = model_card_data['base_model']
	# 			base_model_data = pd.DataFrame(api.list_models(model_name=base_model, full=True))
	# 			model_attributes['modelCreator'] = base_model_data.loc[0]['author']
	# except KeyError:
	# 	pass

	model_attributes['licenseToUse'] = match_license(model_tags)

	model_attributes['libraryFramework'] = [] 
	for t in model_tags:
		if t in tag_library:
			model_attributes['libraryFramework'].append(t)

	model_attributes['contextLength'] = None
	model_attributes['developers'] = [model.author]
	model_attributes['openSource'] = True

	model_attributes['uri'] = match_uri(model_tags)

	model_attributes['fineTuned'] = None # if there is a 'base_model' in card_data, it is fine-tuned
	try:
		if model_card_data is not None:
			if 'base_model' in model_card_data:
				model_attributes['fineTuned'] = True
	except KeyError:
		pass

	model_attributes['carbonEmission [CO2eq tons]'] = None
	try:
		if model_card_data is not None:
			model_attributes['carbonEmission [CO2eq tons]'] = model_card_data['co2_eq_emissions']
	except KeyError:
		pass

	model_attributes['tokenizer'] = None

	return model_attributes

In [ ]:
def add_to_json_file(data, file_path):

    if os.path.exists(file_path):
        with open(file_path, 'r+', encoding='utf-8') as f:

            f.seek(0, os.SEEK_END)
            f.seek(f.tell() - 1, os.SEEK_SET)
            f.truncate()
            f.write(',\n')
            json.dump(data, f, indent=4)
            f.write(']')
    else:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump([data], f, indent=4)

In [ ]:
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
result_path = os.path.join(parent_path, 'database', 'hf_extracted_json')
os.makedirs(result_path, exist_ok=True)
file_path = os.path.join(result_path, 'models_data.json')

# tot 697,162 models
count = 0
start_time = time.time()
for task in TAG_DOWNSTREAM_TASK:
    print(f'Processing {task} models...')
    models = api.list_models(filter=task, full=True, cardData=True)
    for model in models:
        model_attributes = extract_model_attributes_optimized(model)
        add_to_json_file(model_attributes, file_path)
        count += 1
        if count % 1000 == 0:
            print(f'{count} models processed, {time.time() - start_time} seconds elapsed')

In [2]:
with open('/Users/alessiobuda/Documents/GitHub/chatIMPACT/database/hf_extracted_json/models_data.json', 'r') as file:
    data = json.load(file)
models_df = pd.DataFrame(data)

In [9]:
models_df.head(10)

,name,id,version,numberOfParameters,quantization,architecture,languages,modelCreator,licenseToUse,libraryFramework,contextLength,developers,openSource,uri,fineTuned,carbonEmission [CO2eq tons],tokenizer
0,ctrl,Salesforce/ctrl,None,None,None,None,[en],None,bsd-3-clause,"[transformers, pytorch, tf]",None,[Salesforce],True,1909.05858,None,None,None
1,distilgpt2,distilbert/distilgpt2,None,None,None,None,[en],None,apache-2.0,"[transformers, pytorch, tf, jax, rust, safeten...",None,[distilbert],True,1910.01108,None,149200,None
2,gpt2-large,openai-community/gpt2-large,None,None,None,None,[en],None,mit,"[transformers, pytorch, tf, jax, rust, onnx, s...",None,[openai-community],True,1910.09700,None,None,None
3,gpt2-medium,openai-community/gpt2-medium,None,None,None,None,[en],None,mit,"[transformers, pytorch, tf, jax, rust, onnx, s...",None,[openai-community],True,1910.09700,None,None,None
4,gpt2-xl,openai-community/gpt2-xl,None,None,None,None,[en],None,mit,"[transformers, pytorch, tf, jax, rust, safeten...",None,[openai-community],True,1910.09700,None,None,None
5,gpt2,openai-community/gpt2,None,None,None,None,[en],None,mit,"[transformers, pytorch, tf, jax, rust, onnx, s...",None,[openai-community],True,10.57967/hf/0039,None,None,None
6,openai-gpt,openai-community/openai-gpt,None,None,None,None,[en],None,mit,"[transformers, pytorch, tf, rust, safetensors]",None,[openai-community],True,1705.11168,None,None,None
7,transfo-xl-wt103,transfo-xl/transfo-xl-wt103,None,None,None,None,[en],None,None,"[transformers, pytorch, tf]",None,[transfo-xl],True,1901.02860,None,None,None
8,xlnet-base-cased,xlnet/xlnet-base-cased,None,None,None,None,[en],None,mit,"[transformers, pytorch, tf, rust]",None,[xlnet],True,1906.08237,None,None,None
9,xlnet-large-cased,xlnet/xlnet-large-cased,None,None,None,None,[en],None,mit,"[transformers, pytorch, tf]",None,[xlnet],True,1906.08237,None,None,None


In [3]:
df_filtered = models_df[models_df['architecture'].notna()]

# Process each row
count = 0
start_time = time.time()
for index, row in df_filtered.iterrows():
    # Find the row where 'id' matches the 'architecture' of the current row
    try:
        matching_row = models_df[models_df['id'].astype(str) == str(row['architecture'])]
    except ValueError:
        print("architecture_value: ", row['architecture'])
        print(row)
        print("len(matching_rows): ", len(matching_row))
        print("matching_rows.empty: ", matching_row.empty)
        break
    
    if not matching_row.empty:
        # Get the first developer from the 'developers' list
        first_developer = matching_row['developers'].iloc[0][0] if matching_row['developers'].iloc[0] else None
        
        # Set the 'modelCreator' attribute of the original row
        models_df.at[index, 'modelCreator'] = first_developer
    count += 1
    if count % 1000 == 0:
        print(f'{count} rows processed ({count/len(df_filtered)*100} %), elapsed time: {time.time() - start_time} seconds, estimated time remaining: {(time.time() - start_time) / count * (len(df_filtered) - count)} seconds')
    
        

1000 rows processed (1.826951183864367 %), elapsed time: 24.251325130462646 seconds, estimated time remaining: 1303.1692712688446 seconds
2000 rows processed (3.653902367728734 %), elapsed time: 48.823071241378784 seconds, estimated time remaining: 1287.3668745117186 seconds
3000 rows processed (5.480853551593102 %), elapsed time: 74.07481813430786 seconds, estimated time remaining: 1277.4449837830862 seconds
4000 rows processed (7.307804735457468 %), elapsed time: 98.7923891544342 seconds, estimated time remaining: 1253.0826761312485 seconds
5000 rows processed (9.134755919321835 %), elapsed time: 123.23447012901306 seconds, estimated time remaining: 1225.8379331253052 seconds
6000 rows processed (10.961707103186203 %), elapsed time: 148.04533123970032 seconds, estimated time remaining: 1202.5229740460713 seconds
7000 rows processed (12.78865828705057 %), elapsed time: 172.9973590373993 seconds, estimated time remaining: 1179.743168770654 seconds
8000 rows processed (14.61560947091493

In [5]:
models_list = models_df.to_dict(orient='records')

In [6]:
with open("/Users/alessiobuda/Documents/GitHub/chatIMPACT/database/hf_extracted_json/models_data_with_creator.json", "w") as json_file:
    json.dump(models_list, json_file, indent=4)

In [7]:
# Remove 'id' field if it exists
for model in models_list:
    model.pop('id', None)

# Write to JSON file
with open("/Users/alessiobuda/Documents/GitHub/chatIMPACT/database/hf_extracted_json/models_data_with_creator_no_id.json", "w") as json_file:
    json.dump(models_list, json_file, indent=4)

### Statistics

In [22]:
# Read JSON and count available attributes

def models_statistics(file_name):

	name_count = 0
	version_count = 0
	number_of_parameters_count = 0
	quantization_count = 0
	architecture_count = 0
	languages_count = 0
	model_creator_count = 0
	license_count = 0
	library_count = 0
	context_length_count = 0
	developers_count = 0
	open_source_count = 0
	uri_count = 0
	finetuned_count = 0
	carbon_emission_count = 0
	tokenizer_count = 0

	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'HF entries', 'hf extracted json')

	models_json = open(os.path.join(result_path, file_name))
	models_data_json = json.load(models_json)

	models_df = pd.DataFrame(models_data_json) 

	# TODO: add more attributes (?)
	for idx, item in enumerate(models_data_json):
		if item['name'] is not None:
			name_count += 1
		if item['version'] is not None:
			version_count += 1
		if item['numberOfParameters'] is not None:
			number_of_parameters_count += 1
		if item['quantization'] is not None:
			quantization_count += 1
		if item['architecture'] is not None:
			architecture_count += 1
		if len(item['languages']) > 0:
			languages_count += 1
		if item['modelCreator'] is not None:
			model_creator_count += 1
		if item['licenseToUse'] is not None:
			license_count += 1
		if len(item['libraryFramework']) > 0:
			library_count += 1
		if item['contextLength'] is not None:
			context_length_count += 1
		if len(item['developers']) > 0:
			developers_count += 1
		if item['openSource'] is not None:
			open_source_count += 1
		if item['uri'] is not None:
			uri_count += 1
		if item['fineTuned'] is not None:
			finetuned_count += 1
		if item['carbonEmission [CO2eq tons]'] is not None:
			carbon_emission_count += 1
		if item['tokenizer'] is not None:
			tokenizer_count += 1
	
	total_models = idx + 1

	print(f'Number of processed models: {total_models}')
	print(f'    Name: {name_count} ({(name_count / total_models) * 100:.2f}%)')
	print(f'    Version: {version_count} ({(version_count / total_models) * 100:.2f}%)')
	print(f'    Number of Parameters: {number_of_parameters_count} ({(number_of_parameters_count / total_models) * 100:.2f}%)')
	print(f'    Quantization: {quantization_count} ({(quantization_count / total_models) * 100:.2f}%)')
	print(f'    Architecture: {architecture_count} ({(architecture_count / total_models) * 100:.2f}%)')
	print(f'    Languages: {languages_count} ({(languages_count / total_models) * 100:.2f}%)')
	print(f'    Model creator: {model_creator_count} ({(model_creator_count / total_models) * 100:.2f}%)')
	print(f'    License to use: {license_count} ({(license_count / total_models) * 100:.2f}%)')
	print(f'    Library: {library_count} ({(library_count / total_models) * 100:.2f}%)')
	print(f'    Context Length: {context_length_count} ({(context_length_count / total_models) * 100:.2f}%)')
	print(f'    Developers: {developers_count} ({(developers_count / total_models) * 100:.2f}%)')
	print(f'    Open Source: {open_source_count} ({(open_source_count / total_models) * 100:.2f}%)')
	print(f'    URI: {uri_count} ({(uri_count / total_models) * 100:.2f}%)')
	print(f'    Fine-tuned: {finetuned_count} ({(finetuned_count / total_models) * 100:.2f}%)')
	print(f'    Carbon emission: {carbon_emission_count} ({(carbon_emission_count / total_models) * 100:.2f}%)')
	print(f'    Tokenizer: {tokenizer_count} ({(tokenizer_count / total_models) * 100:.2f}%)')

	availability = pd.DataFrame(columns=['id', 'entity name', 'attribute name', 'available API', 'available scraping'])

	llm_attributes = models_data_json[0].keys()

	for idx, item in enumerate(models_data_json):
		model_name = item['name']
		for attr in llm_attributes:
			if item[attr] is not None and type(item[attr]) != list:
				availability = pd.concat([availability, pd.DataFrame([{'id': model_name, 'entity name': 'LLM', 'attribute name': attr, 'available API': True, 'available scraping': False}])], ignore_index=True) # all llm attributes we are able to extract come from API, no attribute is obtained by scraping
			elif item[attr] is not None and type(item[attr]) == list and len(item[attr]) > 0:
				availability = pd.concat([availability, pd.DataFrame([{'id': model_name, 'entity name': 'LLM', 'attribute name': attr, 'available API': True, 'available scraping': False}])], ignore_index=True)
			else:
				availability = pd.concat([availability, pd.DataFrame([{'id': model_name, 'entity name': 'LLM', 'attribute name': attr, 'available API': False, 'available scraping': False}])], ignore_index=True)

	return availability



In [23]:
availability = models_statistics('models_data.json')

Number of processed models: 276660
    Name: 276660 (100.00%)
    Version: 0 (0.00%)
    Number of Parameters: 0 (0.00%)
    Quantization: 14843 (5.37%)
    Architecture: 54736 (19.78%)
    Languages: 59988 (21.68%)
    Model creator: 30901 (11.17%)
    License to use: 116065 (41.95%)
    Library: 274520 (99.23%)
    Context Length: 0 (0.00%)
    Developers: 276660 (100.00%)
    Open Source: 276660 (100.00%)
    URI: 50638 (18.30%)
    Fine-tuned: 54736 (19.78%)
    Carbon emission: 1621 (0.59%)
    Tokenizer: 0 (0.00%)


In [ ]:
availability.head(30)

## Dataset

### Information extraction

In [5]:
datasets = api.list_datasets(full=True)

In [6]:
# TODO run for all datasets
datasets = list(itertools.islice(datasets, 0, 1000))
datasets_df = pd.DataFrame(datasets)
datasets_df.head(10)

,id,author,sha,created_at,last_modified,private,gated,disabled,downloads,likes,paperswithcode_id,tags,card_data,siblings
0,amirveyseh/acronym_identification,amirveyseh,15ef643450d589d5883e289ffadeb03563e80a9e,2022-03-02 23:29:22+00:00,2024-01-09 11:39:57+00:00,False,False,False,206,19,acronym-identification,"[task_categories:token-classification, annotat...",{},None
1,ade-benchmark-corpus/ade_corpus_v2,ade-benchmark-corpus,4ba01c71687dd7c996597042449448ea312126cf,2022-03-02 23:29:22+00:00,2024-01-09 11:42:58+00:00,False,False,False,244,25,None,"[task_categories:text-classification, task_cat...",{},None
2,UCLNLP/adversarial_qa,UCLNLP,c2d5f738db1ad21a4126a144dfbb00cb51e0a4a9,2022-03-02 23:29:22+00:00,2023-12-21 14:20:00+00:00,False,False,False,135,32,adversarialqa,"[task_categories:question-answering, task_ids:...",{},None
3,Yale-LILY/aeslc,Yale-LILY,2305f2e63b68056f9b9037a3805c8c196e0d5581,2022-03-02 23:29:22+00:00,2024-01-09 11:49:13+00:00,False,False,False,132,12,aeslc,"[task_categories:summarization, annotations_cr...",{},None
4,nwu-ctext/afrikaans_ner_corpus,nwu-ctext,445834a997dce8b40e1d108638064381de80c497,2022-03-02 23:29:22+00:00,2024-01-09 11:51:47+00:00,False,False,False,103,6,None,"[task_categories:token-classification, task_id...",{},None
5,fancyzhx/ag_news,fancyzhx,eb185aade064a813bc0b7f42de02595523103ca4,2022-03-02 23:29:22+00:00,2024-03-07 12:02:37+00:00,False,False,False,6975,124,ag-news,"[task_categories:text-classification, task_ids...",{},None
6,allenai/ai2_arc,allenai,210d026faf9955653af8916fad021475a3f00453,2022-03-02 23:29:22+00:00,2023-12-21 15:09:48+00:00,False,False,False,836988,116,None,"[task_categories:question-answering, task_ids:...",{},None
7,google/air_dialogue,google,dbdbe7bcef8d344bc3c68a05600f3d95917d6898,2022-03-02 23:29:22+00:00,2024-03-07 15:22:15+00:00,False,False,False,77,15,None,"[task_categories:text-generation, task_categor...",{},None
8,komari6/ajgt_twitter_ar,komari6,af3f2fa5462ac461b696cb300d66e07ad366057f,2022-03-02 23:29:22+00:00,2024-01-09 11:58:01+00:00,False,False,False,135,4,None,"[task_categories:text-classification, task_ids...",{},None
9,legacy-datasets/allegro_reviews,legacy-datasets,71593d1379934286885c53d147bc863ffe830745,2022-03-02 23:29:22+00:00,2024-01-09 11:59:39+00:00,False,False,False,124,4,allegro-reviews,"[task_categories:text-classification, task_ids...",{},None


In [7]:
datasets_df.columns

Index(['id', 'author', 'sha', 'created_at', 'last_modified', 'private',
       'gated', 'disabled', 'downloads', 'likes', 'paperswithcode_id', 'tags',
       'card_data', 'siblings'],
      dtype='object')

In [8]:
datasets_df.loc[0]

id                                   amirveyseh/acronym_identification
author                                                      amirveyseh
sha                           15ef643450d589d5883e289ffadeb03563e80a9e
created_at                                   2022-03-02 23:29:22+00:00
last_modified                                2024-01-09 11:39:57+00:00
private                                                          False
gated                                                            False
disabled                                                         False
downloads                                                          206
likes                                                               19
paperswithcode_id                               acronym-identification
tags                 [task_categories:token-classification, annotat...
card_data                                                           {}
siblings                                                          None
Name: 

In [9]:
datasets_df.loc[2]['tags']

['task_categories:question-answering',
 'task_ids:extractive-qa',
 'task_ids:open-domain-qa',
 'annotations_creators:crowdsourced',
 'language_creators:found',
 'multilinguality:monolingual',
 'source_datasets:original',
 'language:en',
 'license:cc-by-sa-4.0',
 'size_categories:10K<n<100K',
 'format:parquet',
 'modality:text',
 'library:datasets',
 'library:pandas',
 'library:mlcroissant',
 'library:polars',
 'arxiv:2002.00293',
 'arxiv:1606.05250',
 'region:us']

In [11]:
def match_language(entries):
    return find_all_matches(entries, r'language:(\S+)')

def match_size(entries):
    return match_string(entries, r'size_categories:(\S+)')

In [ ]:
def convert_file_size_to_gb(file_size_str):
    """
    Convert the file size string (e.g., '74.6 kB') to gigabytes (GB).
    """
    file_size_parts = file_size_str.split()
    file_size = float(file_size_parts[0])
    unit = file_size_parts[1]

    conversion_factors = {
        'B': 1 / (1024 ** 3),
        'kB': 1 / (1024 ** 2),
        'MB': 1 / 1024,
        'GB': 1,
        'TB': 1024,
    }

    if unit in conversion_factors:
        return float(file_size * conversion_factors[unit])
    else:
        return None

def extract_file_size(url):
    # Fetch the HTML content from the provided URL
    response = requests.get(url)
    html_content = response.content

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the div containing the "Size of downloaded dataset files:" text
    size_label_div = soup.find('div', string='Size of downloaded dataset files:')

    if size_label_div:
        # Find the next sibling div containing the file size
        size_div = size_label_div.find_next('div')
        if size_div:
            # Extract the file size text
            file_size = size_div.get_text(strip=True)
            return file_size
    else:
        return None


In [ ]:
def extract_datasets_attributes(dataset_idx):

	dataset = datasets_df.loc[dataset_idx]
	dataset_tags = datasets_df.loc[dataset_idx]['tags']
	dataset_attributes = dict()

	dataset_attributes['name'] = extract_name(dataset['id'])
	dataset_attributes['size [GB]'] = None

	url = "https://huggingface.co/datasets/" + dataset['id']
	file_size_str = extract_file_size(url)
	if file_size_str:
		file_size_gb = convert_file_size_to_gb(file_size_str)
		if file_size_gb:
			dataset_attributes['size [GB]'] = file_size_gb

	dataset_attributes['languages'] = match_language(dataset_tags)

	# dataset_attributes['dataset creator'] = dataset['author'] # TODO: add attribute in our model?

	dataset_attributes['licenseToUse'] = match_license(dataset_tags)

	dataset_attributes['domain'] = []
	for t in dataset_tags:
		if t in tag_domain:
			dataset_attributes['domain'].append(t)

	dataset_attributes['uri'] = match_uri(dataset_tags) # TODO: add multiple URIs when available?

	dataset_attributes['fineTuning'] = None

	return dataset_attributes

In [ ]:
def dataset_to_json(datasets_df):
    
    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)

    output = []
    
    start_time = time.time()
    for dataset_idx in range(datasets_df.shape[0]):
        output.append(extract_datasets_attributes(dataset_idx))
        if dataset_idx % 10 == 0:
            print(f'Processed {dataset_idx} datasets, elapsed time: {time.time() - start_time:.2f} seconds')
    
    with open(os.path.join(result_path, 'ChatIMPACT.Dataset.json'), 'w', encoding='utf-8') as f:
        json.dump(output, f, indent=4)

In [ ]:
dataset_to_json(datasets_df)

### Info extraction optimized

In [3]:
datasets = api.list_datasets(full=True)

In [6]:
def extract_datasets_attributes_optimized(dataset):

	dataset_tags = dataset.tags
	dataset_attributes = dict()

	dataset_attributes['name'] = extract_name(dataset.id)
	dataset_attributes['size [GB]'] = match_size(dataset_tags)

	# url = "https://huggingface.co/datasets/" + dataset.id
	# file_size_str = extract_file_size(url)
	# if file_size_str:
	# 	file_size_gb = convert_file_size_to_gb(file_size_str)
	# 	if file_size_gb:
	# 		dataset_attributes['size [GB]'] = file_size_gb

	dataset_attributes['languages'] = match_language(dataset_tags)

	# dataset_attributes['dataset creator'] = dataset['author'] # TODO: add attribute in our model?

	dataset_attributes['licenseToUse'] = match_license(dataset_tags)

	dataset_attributes['domain'] = []
	for t in dataset_tags:
		if t in tag_domain:
			dataset_attributes['domain'].append(t)

	dataset_attributes['uri'] = match_uri(dataset_tags) # TODO: add multiple URIs when available?

	dataset_attributes['fineTuning'] = None

	return dataset_attributes

In [13]:
def add_to_json_file(data, file_path):

    if os.path.exists(file_path):
        with open(file_path, 'r+', encoding='utf-8') as f:

            f.seek(0, os.SEEK_END)
            f.seek(f.tell() - 1, os.SEEK_SET)
            f.truncate()
            f.write(',\n')
            json.dump(data, f, indent=4)
            f.write(']')
    else:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump([data], f, indent=4)

In [14]:
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
result_path = os.path.join(parent_path, 'database', 'hf_extracted_json')
os.makedirs(result_path, exist_ok=True)
file_path = os.path.join(result_path, 'datasets_data.json')

count = 0
start_time = time.time()
for task in TAG_DOWNSTREAM_TASK:
    print(f'Processing {task} datasets...')
    datasets = api.list_datasets(filter=task, full=True)
    for dataset in datasets:
        dataset_attributes = extract_datasets_attributes_optimized(dataset)
        add_to_json_file(dataset_attributes, file_path)
        count += 1
        if count % 1000 == 0:
            print(f'{count} datasets processed, {time.time() - start_time} seconds elapsed, estimated time remaining: {(time.time() - start_time) / count * (199642 - count):.2f} seconds')

Processing sentence-similarity datasets...
Processing summarization datasets...
Processing text-classification datasets...
Processing question-answering datasets...
Processing feature-extraction datasets...
Processing zero-shot-classification datasets...
Processing token-classification datasets...
Processing text-generation datasets...
Processing translation datasets...
Processing fill-mask datasets...
Processing table-question-answering datasets...


### Statistics

In [19]:
# Read JSON and count available attributes

def datasets_statistics(file_name):

	name_count = 0
	size_count = 0
	languages_count = 0
	license_count = 0
	domain_count = 0
	uri_count = 0
	finetuning_count = 0

	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'HF entries', 'hf extracted json')
	print()
	datasets_json = open(os.path.join(result_path, file_name))
	datasets_data_json = json.load(datasets_json)

	for idx, item in enumerate(datasets_data_json):
		if item['name'] is not None:
			name_count += 1
		if item['size [rows]'] is not None: # can be size [GB] or size [rows]
			size_count += 1
		if len(item['languages']) > 0:
			languages_count += 1	
		if item['licenseToUse'] is not None:	
			license_count += 1
		if len(item['domain']) > 0:
			domain_count += 1	
		if item['uri'] is not None:
			uri_count += 1
		if item['fineTuning'] is not None:
			finetuning_count += 1
	
	total_datasets = idx + 1	
	print(f'Number of processed datasets: {total_datasets}')
	print(f'    Name: {name_count} ({(name_count / total_datasets) * 100:.2f}%)')
	print(f'    Size: {size_count} ({(size_count / total_datasets) * 100:.2f}%)')
	print(f'    Languages: {languages_count} ({(languages_count / total_datasets) * 100:.2f}%)')
	print(f'    License to use: {license_count} ({(license_count / total_datasets) * 100:.2f}%)')
	print(f'    Domain: {domain_count} ({(domain_count / total_datasets) * 100:.2f}%)')
	print(f'    URI: {uri_count} ({(uri_count / total_datasets) * 100:.2f}%)')
	print(f'    Fine-tuning: {finetuning_count} ({(finetuning_count / total_datasets) * 100:.2f}%)')

	availability = pd.DataFrame(columns=['id', 'entity name', 'attribute name', 'available API', 'available scraping'])

	datasets_attributes = datasets_data_json[0].keys()

	for idx, item in enumerate(datasets_data_json):
		dataset_name = item['name']
		for attr in datasets_attributes:
			if item[attr] is not None and attr == 'size [GB]':
				availability = pd.concat([availability, pd.DataFrame([{'id': dataset_name, 'entity name': 'Dataset', 'attribute name': attr, 'available API': False, 'available scraping': True}])], ignore_index=True)
			elif item[attr] is not None and type(item[attr]) != list:
				availability = pd.concat([availability, pd.DataFrame([{'id': dataset_name, 'entity name': 'Dataset', 'attribute name': attr, 'available API': True, 'available scraping': False}])], ignore_index=True) # all llm attributes we are able to extract come from API, no attribute is obtained by scraping
			elif item[attr] is not None and type(item[attr]) == list and len(item[attr]) > 0:
					availability = pd.concat([availability, pd.DataFrame([{'id': dataset_name, 'entity name': 'Dataset', 'attribute name': attr, 'available API': True, 'available scraping': False}])], ignore_index=True)
			else:
				availability = pd.concat([availability, pd.DataFrame([{'id': dataset_name, 'entity name': 'Dataset', 'attribute name': attr, 'available API': False, 'available scraping': False}])], ignore_index=True)

	return availability

In [18]:
datasets_statistics('datasets_data.json')


Number of processed datasets: 773
    Name: 773 (100.00%)
    Size: 729 (94.31%)
    Languages: 747 (96.64%)
    License to use: 748 (96.77%)
    Domain: 24 (3.10%)
    URI: 107 (13.84%)
    Fine-tuning: 0 (0.00%)


In [20]:
datasets_statistics('datasets_data.json')


Number of processed datasets: 199654
    Name: 199654 (100.00%)
    Size: 150976 (75.62%)
    Languages: 22559 (11.30%)
    License to use: 54002 (27.05%)
    Domain: 7319 (3.67%)
    URI: 7032 (3.52%)
    Fine-tuning: 0 (0.00%)


In [ ]:
availability.head(20)

## Downstream Task

In [ ]:
def fetch_and_extract_text(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        target_paragraph = soup.find('p', class_='text-[1.2rem] text-gray-500')
        
        if target_paragraph:
            return target_paragraph.get_text().strip()
        else:
            return "Target paragraph not found."
    else:
        return f"Failed to fetch the webpage. Status code: {response.status_code}"

def create_tasks_json():

    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)

    tasks_data = []

    for task in TAG_DOWNSTREAM_TASK:
        url = f"https://huggingface.co/tasks/{task}"
        description = fetch_and_extract_text(url)
        
        tasks_data.append({
            "name": task,
            "description": description, # TODO: text2text generation has no description
            "sub-task": []
        })
        
        print(f"Processed: {task}")
        # time.sleep(0.5)  # Be polite to the server

    with open(result_path + '/ChatIMPACT.DownstreamTask.json', 'w', encoding='utf-8') as f:
        json.dump(tasks_data, f, ensure_ascii=False, indent=2)

In [ ]:
# create_tasks_json()

In [ ]:
# Read JSON and count available attributes

def tasks_statistics():
	name_count = 0
	description_count = 0
	sub_task_count = 0
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')

	task_json = open(os.path.join(result_path, 'ChatIMPACT.DownstreamTask.json'))
	task_data_json = json.load(task_json)

	for idx, item in enumerate(task_data_json):
		if item['name'] is not None:
			name_count += 1	
		if item['description'] is not None:
			description_count += 1
		if len(item['sub-task']) > 0:
			sub_task_count += 1
	
	task_count = idx + 1
	print(f'Number of processed task: {idx + 1}')
	print(f'    Name: {name_count} ({(name_count / task_count) * 100:.2f}%)')
	print(f'    Description: {description_count} ({(description_count / task_count) * 100:.2f}%)')
	print(f'    Sub-task: {sub_task_count} ({(sub_task_count / task_count) * 100:.2f}%)')

	availability = pd.DataFrame(columns=['id', 'entity name', 'attribute name', 'available API', 'available scraping'])

	task_attributes = task_data_json[0].keys()

	for idx, item in enumerate(task_data_json):
		task_name = item['name']
		for attr in task_attributes:
			if item[attr] is not None and type(item[attr]) != list:
					availability = pd.concat([availability, pd.DataFrame([{'id': task_name, 'entity name': 'DownstreamTask', 'attribute name': attr, 'available API': False, 'available scraping': True}])], ignore_index=True) # all llm attributes we are able to extract come from API, no attribute is obtained by scraping
			elif item[attr] is not None and type(item[attr]) == list and len(item[attr]) > 0:
					availability = pd.concat([availability, pd.DataFrame([{'id': task_name, 'entity name': 'DownstreamTask', 'attribute name': attr, 'available API': False, 'available scraping': True}])], ignore_index=True)
			else:
				availability = pd.concat([availability, pd.DataFrame([{'id': task_name, 'entity name': 'DownstreamTask', 'attribute name': attr, 'available API': False, 'available scraping': False}])], ignore_index=True)

	return availability


In [ ]:
availability = tasks_statistics()

In [ ]:
availability.head(33)

# Metrics

In [14]:
# Scrape metrics and descriptions from HF

url_metrics = 'https://huggingface.co/metrics'

response = requests.get(url_metrics)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

h4_tags = soup.find_all('h4')
metrics = [h4_tag.get_text(strip=True) for h4_tag in h4_tags]
# print(metrics)

p_tags = soup.find_all('p')
descriptions = [p_tag.get_text() for p_tag in p_tags]
descriptions = descriptions[2:] # drop first lines
# print(descriptions)

# remove from the list the metrics withoud description (not useful for our purpose)
metrics.remove('AlhitawiMohammed22/CER_Hu-Evaluation-Metrics')
metrics.remove('Aye10032/loss_metric')
metrics.remove('giulio98/code_eval_outputs')
metrics.remove('maysonma/lingo_judge_metric')
metrics.remove('lvwerra/test')
metrics.remove('sma2023/wil')


assert len(metrics) == len(descriptions)
print(len(metrics))

268


In [15]:
# From the lists, remove the descriptions and then the relative metric in the same index that have in the description 'TODO: add a description here\n\t\t\t\t\t\t'ArithmeticError

for i, description in enumerate(descriptions):
    if 'TODO: add a description here' in description:
        metrics.pop(i)
        descriptions.pop(i)

assert len(metrics) == len(descriptions)
print(len(metrics))

221


In [16]:
def create_metrics_json(metrics, descriptions):

    metrics_data = []

    current_path = os.getcwd()
    parent_path = os.path.dirname(current_path)
    result_path = os.path.join(parent_path, 'database', 'hf extracted json')
    
    os.makedirs(result_path, exist_ok=True)
    
    for idx in range(len(metrics)):
        metric_attributes = dict()

        metric_attributes['name'] = metrics[idx]
        metric_attributes['description'] = descriptions[idx]
        metric_attributes['trained'] = None
        metric_attributes['context'] = None
        metric_attributes['featureBased/endToEnd'] = None
        metric_attributes['granularity'] = None

    
        metrics_data.append(metric_attributes)
        

    with open(os.path.join(result_path, 'ChatIMPACT.Metric.json'), 'w', encoding='utf-8') as f:
        json.dump(metrics_data, f, indent=4)

In [17]:
create_metrics_json(metrics, descriptions)

In [ ]:
# Read JSON and count available attributes

def metric_statistics():
	name_count = 0
	description_count = 0
	context_count = 0
	featureBased_endToEnd_count = 0
	granularity_count = 0
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')

	metric_json = open(os.path.join(result_path, 'ChatIMPACT.Metric.json'))
	metric_data_json = json.load(metric_json)

	for idx, item in enumerate(metric_data_json):
		if item['name'] is not None:
			name_count += 1
		if item['description'] is not None:
			description_count += 1
		if item['context'] is not None:
			context_count += 1
		if item['featureBased/endToEnd'] is not None:
			featureBased_endToEnd_count += 1
		if item['granularity'] is not None:	
			granularity_count += 1
	
	total_datasets = idx + 1

	print(f'Number of processed datasets: {total_datasets}')
	print(f'    Name: {name_count} ({(name_count / total_datasets) * 100:.2f}%)')
	print(f'    Description: {description_count} ({(description_count / total_datasets) * 100:.2f}%)')
	print(f'    Context: {context_count} ({(context_count / total_datasets) * 100:.2f}%)')
	print(f'    FeatureBased/endToEnd: {featureBased_endToEnd_count} ({(featureBased_endToEnd_count / total_datasets) * 100:.2f}%)')
	print(f'    Granularity: {granularity_count} ({(granularity_count / total_datasets) * 100:.2f}%)')
	
	availability = pd.DataFrame(columns=['id', 'entity name', 'attribute name', 'available API', 'available scraping'])

	metric_attributes = metric_data_json[0].keys()

	for idx, item in enumerate(metric_data_json):
		task_name = item['name']
		for attr in metric_attributes:
			if item[attr] is not None and type(item[attr]) != list:
					availability = pd.concat([availability, pd.DataFrame([{'id': task_name, 'entity name': 'Metric', 'attribute name': attr, 'available API': False, 'available scraping': True}])], ignore_index=True) # all llm attributes we are able to extract come from API, no attribute is obtained by scraping
			elif item[attr] is not None and type(item[attr]) == list and len(item[attr]) > 0:
					availability = pd.concat([availability, pd.DataFrame([{'id': task_name, 'entity name': 'Metric', 'attribute name': attr, 'available API': False, 'available scraping': True}])], ignore_index=True)
			else:
				availability = pd.concat([availability, pd.DataFrame([{'id': task_name, 'entity name': 'Metric', 'attribute name': attr, 'available API': False, 'available scraping': False}])], ignore_index=True)

	return availability


In [ ]:
availability = metric_statistics()

In [ ]:
availability.head(10)

# Train relationship

In [ ]:
def extract_train_relationship():
    
    train_relationship_list = []
    
    for model_idx in range(len(models_df)):
        
        model = models_df.loc[model_idx]
        model_tags = models_df.loc[model_idx]['tags']
        datasets = match_dataset(model_tags)
        
        if not datasets:
            continue
        else:
            train_relationship = dict()
            train_relationship['Models'] = extract_name(model['id']) # {"$oid": <...>} for MongoDB
            train_relationship['Datasets'] = datasets # [{"$oid": <...>}, ..., {"$oid": <...>}] for MongoDB
            train_relationship_list.append(train_relationship)
    
    return train_relationship_list

In [ ]:
def create_train_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	train_relationship = extract_train_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.TrainRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(train_relationship, f, indent=4)

In [ ]:
create_train_relationship_json()

# SuitedFor relationship

In [ ]:
def extract_suited_for_relationship():
    
    suited_for_relationship_list = []
    
    for model_idx in range(len(models_df)):
        
        model = models_df.loc[model_idx]
        model_tags = models_df.loc[model_idx]['tags']
        
        tasks = []
        for t in model_tags:
            if t in TAG_DOWNSTREAM_TASK:
                tasks.append(t)

        if not tasks:
            continue
        else:
            suited_for_relationship = dict()
            suited_for_relationship['LargeLanguageModel'] = extract_name(model['id']) # {"$oid": <...>} for MongoDB
            suited_for_relationship['DownstreamTask'] = tasks # [{"$oid": <...>}, ..., {"$oid": <...>}] for MongoDB
            suited_for_relationship_list.append(suited_for_relationship)
    
    return suited_for_relationship_list

In [ ]:
def create_suited_for_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	suited_for_relationship = extract_suited_for_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.SuitedForRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(suited_for_relationship, f, indent=4)

In [ ]:
create_suited_for_relationship_json()

# Enable relationship

In [ ]:
def match_tasks(entries):
    return find_all_matches(entries, r'task_categories:(\S+)')

In [ ]:
def extract_enable_relationship():
	
	enable_relationship_list = []
	
	for dataset_idx in range(len(datasets_df)):
		
		dataset = datasets_df.loc[dataset_idx]
		dataset_tags = datasets_df.loc[dataset_idx]['tags']
		
		tasks = match_tasks(dataset_tags)

		if not tasks:
			continue
		else:
			enable_relationship = dict()
			enable_relationship['Dataset'] = extract_name(dataset['id']) # {"$oid": <...>} for MongoDB
			enable_relationship['DownstreamTask'] = tasks # [{"$oid": <...>}, ..., {"$oid": <...>}] for MongoDB
			enable_relationship_list.append(enable_relationship)
	
	return enable_relationship_list

In [ ]:
def create_enable_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	enable_relationship = extract_enable_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.EnableRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(enable_relationship, f, indent=4)

In [ ]:
create_enable_relationship_json()

# Assess relationship

In [ ]:
# TODO: here https://huggingface.co/tasks some tasks have associated metrics, we could scrape the tasks one by one

def extract_assess_relationship():

    assess = []
    for task in TAG_DOWNSTREAM_TASK:
        assess_element = {'Metric': [], 'DownstreamTask': task}
        print(f"Processing task: {task}\n")
        url = f"https://huggingface.co/tasks/{task}"
        # Fetch the webpage
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            return

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract all the <dl> elements
        dl_elements = soup.find_all('dl', class_='flex items-center rounded-lg border border-gray-100')

        # Loop through each <dl> element
        for dl in dl_elements:
            # Extract the metric name from the <dt> tag inside the <summary>
            metric_name = dl.find('dt').get_text(strip=True)

            assess_element['Metric'].append(metric_name)

        assess.append(assess_element)
    return assess


In [ ]:
def create_asess_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	assess_relationship = extract_assess_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.AssessRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(assess_relationship, f, indent=4)

In [ ]:
create_asess_relationship_json()

# Evaluate relationship

In [ ]:
# TODO: check that this is correct (the output and the model cards on HF do not seem to be coherent?)
# Model card template: https://github.com/huggingface/hub-docs/blob/main/modelcard.md?plain=1

def extract_evaluate_relationship():

	evaluate_relationship_list = []

	for model_idx in range(len(models_df)):

		model = models_df.loc[model_idx]
		
		try:
			model_card_data = next(api.list_models(model_name=model['id'], full=True, cardData=True)).card_data.to_dict()
		except AttributeError:
			print('No card data available for this model')
		
		metrics = []
		if model_card_data is not None:
			if 'metrics' in model_card_data:
				metrics = model_card_data['metrics']
			
		if not metrics:
			continue
		else:
			evaluate_relationship = dict()
			evaluate_relationship['LargeLanguageModel'] = extract_name(model['id'])
			evaluate_relationship['Metric'] = metrics
			evaluate_relationship_list.append(evaluate_relationship)
	
	return evaluate_relationship_list

In [ ]:
def create_evaluate_relationship_json():
	
	current_path = os.getcwd()
	parent_path = os.path.dirname(current_path)
	result_path = os.path.join(parent_path, 'database', 'hf extracted json')
	
	os.makedirs(result_path, exist_ok=True)

	evaluate_relationship = extract_evaluate_relationship()

	with open(os.path.join(result_path, 'ChatIMPACT.EvaluateRelationship.json'), 'w', encoding='utf-8') as f:
		json.dump(evaluate_relationship, f, indent=4)

In [ ]:
create_evaluate_relationship_json()